In [1]:
!pip install more_polars

In [2]:
!pip install more_polars --upgrade

## Scaling up of Select, Drop, and Rename with Comprehensions

In [9]:
import polars as pl
import re

## Outline

* Using a `dict` comprehension to clean column names,
* Using a `list` comprehension to drop columns,
* 
* `select` helper functions
* Basic `filter` format

## Example - Health Survey

In [10]:
survey_raw = pl.read_csv("./data/health_survey.csv")
survey_raw.columns

['',
 'F1',
 'F5',
 'F2',
 'F1.1',
 'F2.1',
 'F6',
 'F4',
 'F3',
 'F5.1',
 'F1.2',
 'F2.2',
 'F6.1',
 'F2.3',
 'F4.1',
 'F2.4',
 'F5.2',
 'F2.5',
 'F6.2',
 'F1.3',
 'F2.6',
 'F5.3',
 'F4.2',
 'F2.7',
 'F3.1',
 'F2.8',
 'F5.4',
 'F3.2',
 'F1.4',
 'F3.3',
 'F1.5',
 'F5.5',
 'F6.3',
 'F1.6',
 'F5.6',
 'F2.9',
 'F3.4',
 'F4.3',
 'F2.10',
 'F1.7',
 'F6.4',
 'F4.4',
 'F5.7',
 'F3.5',
 'F2.11']

### Example - Mexico's healthcare spending

In [11]:
mexico_hc_spending = pl.read_csv("https://github.com/yardsale8/DSCI_210_R_notebooks/raw/main/data/Mexico_Health_Spending.csv")

mexico_hc_spending.head()

Year,Type,"Spending (in billions, 2017 Purchasing Power Parity Dollars PPP $)"
i64,str,f64
1995,"""Government""",25.1
1996,"""Government""",25.7
1997,"""Government""",28.4
1998,"""Government""",31.2
1999,"""Government""",34.4


## Renaming columns

* Use `rename` method to rename one or more columns.
* Use a `dict` of pair, i.e. `{'old_name_1':'new_name_1', ...}`

In [13]:
survey = survey_raw.rename({'':'ID'})
survey.head()

ID,F1,F5,F2,F1.1,F2.1,F6,F4,F3,F5.1,F1.2,F2.2,F6.1,F2.3,F4.1,F2.4,F5.2,F2.5,F6.2,F1.3,F2.6,F5.3,F4.2,F2.7,F3.1,F2.8,F5.4,F3.2,F1.4,F3.3,F1.5,F5.5,F6.3,F1.6,F5.6,F2.9,F3.4,F4.3,F2.10,F1.7,F6.4,F4.4,F5.7,F3.5,F2.11
i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
1,"""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree"""
2,"""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Neither Agree nor Disagree""","""Neither Agree nor Disagree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree"""
3,"""Strongly Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Strongly Agree""","""Strongly Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Strongly Agree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Disagree""","""Strongly Agree""","""Somewhat Agree""","""Strongly Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Strongly Agree""","""Somewhat Disagree""","""Somewhat Disagree""","""Strongly Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Strongly Agree""","""Strongly Disagree""","""Somewhat Agree"""
4,"""Somewhat Agree""","""Somewhat Agree""","""Strongly Agree""","""Somewhat Agree""","""Strongly Agree""","""Neither Agree nor Disagree""","""Neither Agree nor Disagree""","""Somewhat Disagree""","""Somewhat Agree""","""Strongly Agree""","""Strongly Agree""","""Somewhat Agree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Strongly Agree""","""Somewhat Disagree""","""Strongly Agree""","""Somewhat Agree""","""Somewhat Agree""","""Strongly Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Some

## Renaming many columns with a `dict` comprehension

In [16]:
fix_name = lambda n: n.lower().replace('.', '_')

{n:fix_name(n) for n in survey.columns}

{'ID': 'id',
 'F1': 'f1',
 'F5': 'f5',
 'F2': 'f2',
 'F1.1': 'f1_1',
 'F2.1': 'f2_1',
 'F6': 'f6',
 'F4': 'f4',
 'F3': 'f3',
 'F5.1': 'f5_1',
 'F1.2': 'f1_2',
 'F2.2': 'f2_2',
 'F6.1': 'f6_1',
 'F2.3': 'f2_3',
 'F4.1': 'f4_1',
 'F2.4': 'f2_4',
 'F5.2': 'f5_2',
 'F2.5': 'f2_5',
 'F6.2': 'f6_2',
 'F1.3': 'f1_3',
 'F2.6': 'f2_6',
 'F5.3': 'f5_3',
 'F4.2': 'f4_2',
 'F2.7': 'f2_7',
 'F3.1': 'f3_1',
 'F2.8': 'f2_8',
 'F5.4': 'f5_4',
 'F3.2': 'f3_2',
 'F1.4': 'f1_4',
 'F3.3': 'f3_3',
 'F1.5': 'f1_5',
 'F5.5': 'f5_5',
 'F6.3': 'f6_3',
 'F1.6': 'f1_6',
 'F5.6': 'f5_6',
 'F2.9': 'f2_9',
 'F3.4': 'f3_4',
 'F4.3': 'f4_3',
 'F2.10': 'f2_10',
 'F1.7': 'f1_7',
 'F6.4': 'f6_4',
 'F4.4': 'f4_4',
 'F5.7': 'f5_7',
 'F3.5': 'f3_5',
 'F2.11': 'f2_11'}

In [19]:
fix_name = lambda n: n.lower().replace('.', '_')

survey = (survey_raw 
          .rename({'':'ID'})
          .rename({n:fix_name(n) for n in survey_raw.columns}) 
         )
survey.head(2)

ID,f1,f5,f2,f1_1,f2_1,f6,f4,f3,f5_1,f1_2,f2_2,f6_1,f2_3,f4_1,f2_4,f5_2,f2_5,f6_2,f1_3,f2_6,f5_3,f4_2,f2_7,f3_1,f2_8,f5_4,f3_2,f1_4,f3_3,f1_5,f5_5,f6_3,f1_6,f5_6,f2_9,f3_4,f4_3,f2_10,f1_7,f6_4,f4_4,f5_7,f3_5,f2_11
i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
1,"""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree"""
2,"""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Neither Agree nor Disagree""","""Neither Agree nor Disagree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree"""


## Cleaning up all whitespace and punctuation.

It is often useful to convert column names to proper variable names (in fact R does this by default, which is why you see so many dots).  While a complete function for fixing names is beyond the scope of this notebook (see [the `R` `make.names`](https://stat.ethz.ch/R-manual/R-devel/library/base/html/make.names.html) for some hints on what it takes), let's make a function for removing whitespace and punctuation.

#### Step 1 - Play around and build a function

In [23]:
from composable import pipeable
from string import whitespace, punctuation

punctuation = punctuation.replace('_', '') # Underscores are allowed

chr_to_remove = f'[{whitespace + punctuation}]'
chr_to_remove

'[ \t\n\r\x0b\x0c!"#$%&\'()*+,-./:;<=>?@[\\]^`{|}~]'

In [24]:
replace_bad_char = pipeable(lambda s: re.sub(chr_to_remove, '_', s))

In [25]:
[replace_bad_char(c) for c in survey_raw.columns]

['',
 'F1',
 'F5',
 'F2',
 'F1_1',
 'F2_1',
 'F6',
 'F4',
 'F3',
 'F5_1',
 'F1_2',
 'F2_2',
 'F6_1',
 'F2_3',
 'F4_1',
 'F2_4',
 'F5_2',
 'F2_5',
 'F6_2',
 'F1_3',
 'F2_6',
 'F5_3',
 'F4_2',
 'F2_7',
 'F3_1',
 'F2_8',
 'F5_4',
 'F3_2',
 'F1_4',
 'F3_3',
 'F1_5',
 'F5_5',
 'F6_3',
 'F1_6',
 'F5_6',
 'F2_9',
 'F3_4',
 'F4_3',
 'F2_10',
 'F1_7',
 'F6_4',
 'F4_4',
 'F5_7',
 'F3_5',
 'F2_11']

In [27]:
[replace_bad_char(c) for c in mexico_hc_spending.columns]

['Year',
 'Type',
 'Spending__in_billions__2017_Purchasing_Power_Parity_Dollars_PPP___']

#### Use piping to make it composable

In [29]:
[(c
  >> replace_bad_char
 ) 
 for c in mexico_hc_spending.columns]

['Year',
 'Type',
 'Spending__in_billions__2017_Purchasing_Power_Parity_Dollars_PPP___']

#### Step 2 - Fixing problems with multiple/leading/trailing `_`

In [30]:
remove_duplicate_underscore = pipeable(lambda s: re.sub(r'_+', '_', s))

In [32]:
[(c
  >> replace_bad_char
  >> remove_duplicate_underscore
 ) 
 for c in mexico_hc_spending.columns]

['Year',
 'Type',
 'Spending_in_billions_2017_Purchasing_Power_Parity_Dollars_PPP_']

In [33]:
remove_outer_underscores = pipeable(lambda s: re.sub(r'(^_|_$)', '', s))

In [35]:
[(c
  >> replace_bad_char
  >> remove_duplicate_underscore
  >> remove_outer_underscores
 ) 
 for c in mexico_hc_spending.columns]


['Year',
 'Type',
 'Spending_in_billions_2017_Purchasing_Power_Parity_Dollars_PPP']

#### Step 3 - Composing into a single function

In [37]:
clean_name = lambda col: (col
                          >> replace_bad_char
                          >> remove_duplicate_underscore
                          >> remove_outer_underscores
                         ) 

#### Step 4 - Using with a `dict` comprehension to rename all columns

In [38]:
(survey_raw
 .rename({col:clean_name(col) for col in survey_raw.columns})
 .head(2)
)

,F1,F5,F2,F1_1,F2_1,F6,F4,F3,F5_1,F1_2,F2_2,F6_1,F2_3,F4_1,F2_4,F5_2,F2_5,F6_2,F1_3,F2_6,F5_3,F4_2,F2_7,F3_1,F2_8,F5_4,F3_2,F1_4,F3_3,F1_5,F5_5,F6_3,F1_6,F5_6,F2_9,F3_4,F4_3,F2_10,F1_7,F6_4,F4_4,F5_7,F3_5,F2_11
i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
1,"""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree"""
2,"""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Neither Agree nor Disagree""","""Neither Agree nor Disagree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree"""


In [40]:
(mexico_hc_spending
 .rename({col:clean_name(col) for col in mexico_hc_spending.columns})
)

Year,Type,Spending_in_billions_2017_Purchasing_Power_Parity_Dollars_PPP
i64,str,f64
1995,"""Government""",25.1
1996,"""Government""",25.7
1997,"""Government""",28.4
1998,"""Government""",31.2
1999,"""Government""",34.4
…,…,…
2011,"""Out-of-pocket""",53.2
2012,"""Out-of-pocket""",54.7
2013,"""Out-of-pocket""",55.2


#### Step 5 - Another layer of abstraction

In [43]:
clean_all_names = lambda df: {col:clean_name(col) for col in df.columns} 

In [44]:
(survey_raw
 .rename(clean_all_names(survey_raw))
 .head(2)
)

,F1,F5,F2,F1_1,F2_1,F6,F4,F3,F5_1,F1_2,F2_2,F6_1,F2_3,F4_1,F2_4,F5_2,F2_5,F6_2,F1_3,F2_6,F5_3,F4_2,F2_7,F3_1,F2_8,F5_4,F3_2,F1_4,F3_3,F1_5,F5_5,F6_3,F1_6,F5_6,F2_9,F3_4,F4_3,F2_10,F1_7,F6_4,F4_4,F5_7,F3_5,F2_11
i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
1,"""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree"""
2,"""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Neither Agree nor Disagree""","""Neither Agree nor Disagree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree"""


In [45]:
(mexico_hc_spending
 .rename(clean_all_names(mexico_hc_spending))
 .head(2)
)

Year,Type,Spending_in_billions_2017_Purchasing_Power_Parity_Dollars_PPP
i64,str,f64
1995,"""Government""",25.1
1996,"""Government""",25.7


## <font color="red"> Exercise 3.3 </font>

Your goal is to create a sequence of functions that will reduce a column name containing spaces to the first word of that column. 

To do this, write each of the following functions.

1. A `lambda` function named `get_first_word` that takes a column name, splits it, and keeps only the first name.
2. A `lambda` function named `all_first_words` that takes a data frame and returns a `dict` comprehension used to rename each column to the first word in that column.

In [36]:
## Your code here